# 노가다 이후 임베딩
- **환경설정**은 경로를 설정하는 chunk(cd/.....)까지 돌려주세요.
- 그 이후론 맨 마지막 파트인 **'환경설정 후 이것만 돌려주세요!'** 만 실행해주시면 됩니다!

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 993 kB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 448 kB 76.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


# 환경설정

In [ ]:
import pandas as pd
import json
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import torch
import re
import os
#from konlpy.tag import Okt
import konlpy

from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter("ignore")

from gensim.models import word2vec
from gensim.models import Word2Vec

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 원하는 경로 설정
cd /content/drive/Shareddrives/DSL_2021_2_웰로/(최종)웰로 전달 C팀

/content/drive/Shareddrives/DSL_2021_2_웰로/(최종)웰로 전달 C팀


# 데이터 정의

In [ ]:
df = pd.read_excel('./data/policy_metatable.xlsx') #8만개

In [ ]:
df.index = df.정책ID

In [ ]:
title = df.서비스명
purpose = df.서비스목적
content = df.지원내용
subject = df.지원대상

# 텍스트 전처리

## 사용자 사전 생성
- 지원대상과 관련된 여러 키워드들을 사용자 사전에 등록

### 태그 검색 키워드 추가

In [ ]:
tag_meta = pd.read_excel('./data/tag_meta_subject_final.xlsx')

In [ ]:
tag_meta.head()

,칼럼,카테고리,찾는 단어
0,태그-대상특성,장애인,"장애인,장애"
1,태그-대상특성,농축수산인,"농축수산,농업,농민,농촌,농가,농지,작물,재배,원예특용,어업,양식업,농지소유자,지주..."
2,태그-대상특성,"질병,부상,질환","질병,부상,질환,환자,요양,치매,입원,결핵,아토피,천식,에이즈,성병,AIDS,암환자..."
3,태그-대상특성,난임,"난임,불임"
4,태그-대상특성,임신부,"임신부,임부,출산전,출산예정,예비부모,임신중"


In [ ]:
userdic = []

for Words in tag_meta.카테고리:
  for word in Words.split(','):
    userdic.append(word.strip())

for Words in tag_meta['찾는 단어']:
  for word in Words.split(','):
    userdic.append(word.strip())

userdic = sorted(list(set(userdic)))

In [ ]:
userdic

['1인가구',
 '2자녀',
 '30인미만',
 '3자녀',
 '4자녀',
 '50인미만',
 '518민주유공자',
 '5자녀',
 'AIDS',
 '가축',
 '결핵',
 '결혼장려',
 '경관농업',
 '경관농업인',
 '경관작물',
 '경로당',
 '경종농가',
 '경종농업',
 '경종농업인',
 '경증장애',
 '고기잡이배',
 '고깃배',
 '고엽제',
 '과수농업',
 '과수농업인',
 '구인',
 '구직',
 '구직자',
 '구직활동',
 '구획어업',
 '국가유공자',
 '근로복지',
 '근로자',
 '근로지원',
 '근해어업',
 '기관',
 '기업',
 '기초생활',
 '기초생활수급자',
 '기타농업',
 '기타농업인',
 '기타어업',
 '기타어업인',
 '난임',
 '난치성질환',
 '난치질환',
 '난치질환자',
 '남성',
 '농가',
 '농민',
 '농업',
 '농지',
 '농지소유자',
 '농촌',
 '농축수산',
 '농축수산인',
 '다둥이',
 '다문화',
 '다문화가정',
 '다문화가족',
 '다자녀',
 '다자녀가구',
 '다자녀가정',
 '단체',
 '대학',
 '대학교',
 '대학생',
 '대학원생',
 '독거노인',
 '독립유공자',
 '동아리',
 '모자보건수첩',
 '민간업',
 '민주유공자',
 '벌채',
 '법인',
 '보육',
 '보육시설',
 '보훈가족',
 '보훈대상자',
 '부상',
 '부상자',
 '북한이탈주민',
 '불임',
 '사회복지법인',
 '사회적기업',
 '산모',
 '산후조리원',
 '성병',
 '센터',
 '소사육',
 '소상공인',
 '소상인',
 '수급자',
 '수유부',
 '시설',
 '식물재배',
 '신생아',
 '실업자',
 '심하지않은장애',
 '심한장애',
 '아토피',
 '알선',
 '암환자',
 '양식업',
 '양식업인',
 '양육수당',
 '어린이집',
 '어선소유',
 '어선소유자',
 '어선을',
 '어업',
 '어촌',
 '에이즈',
 '여성',
 '연근해구획어업인

### 노가다 결과 추가


In [ ]:
# sentence_raw='근로 자녀 장려금 근로자 녀 장려 세제 저소득 계층 빈곤 탈출 지원 사회 안전망 확..'
# word = '근로자녀'

# for i in range(len(word)):
#   fix_word = word[:i] + ' ' + word[i:]
#   if fix_word in sentence_raw:
#     sentence_new = re.sub(fix_word,word,sentence_raw) # pattern, replace, subject
#     print(sentence_new)

In [ ]:
# worksheet = gc.open('noun_unique_final').sheet1
# rows = worksheet.get_all_values()

In [ ]:
# more_user_df = pd.DataFrame.from_records(rows[1:], columns = rows[0])
# more_user = more_user_df['추가할 단어'].unique().tolist()

In [ ]:
# more_user.remove('')

In [ ]:
# more_user_lst = []

# for Words in more_user:
#   for word in Words.split(','):
#     more_user_lst.append(word.strip())

In [ ]:
# more_user_lst = list(set(more_user_lst))

In [ ]:
# temp = pd.DataFrame(more_user_lst)
# temp.to_csv('./data/사용자 사전 후보_노가다.csv', index=False, encoding='CP949')

In [ ]:
more_userdic = pd.read_csv('./data/more_userdic.csv',encoding='CP949').word.tolist()

In [ ]:
more_userdic[:5]

['고속도로통행료', '전통시장', '독서문화프로그램', '의료보호혜택', '독립유공']

### 모든 단어 병합 및 사전 형성

In [ ]:
final_userdic = list(set(userdic + more_userdic))

In [ ]:
# final_userdic

In [ ]:
# 코모란에 넣을 사용자사전을 txt 파일로 변경

with open('./data/userdict.txt','w') as f:
  for word in final_userdic:
    f.write(word+'\tNNG\n')

## 품사 태깅

In [ ]:
komoran = konlpy.tag.Komoran(userdic='./data/userdict.txt')

In [ ]:
# 정상적으로 사용자 사전을 인식하는지 테스트
temp = ' '.join(final_userdic)
test = komoran.pos(temp)
test

[('민주유공자', 'NNG'),
 ('다문화가족지원센터', 'NNG'),
 ('꿈나무', 'NNG'),
 ('소귀표', 'NNG'),
 ('농지소유자', 'NNG'),
 ('버스정류장', 'NNG'),
 ('출생신고', 'NNG'),
 ('보호시설', 'NNG'),
 ('중독관리', 'NNG'),
 ('자살예방', 'NNG'),
 ('미아방지', 'NNG'),
 ('건강벼', 'NNG'),
 ('기회부족', 'NNG'),
 ('재능기부', 'NNG'),
 ('종량제 봉투', 'NNG'),
 ('행정서비스', 'NNG'),
 ('방위협의회', 'NNG'),
 ('무료진료', 'NNG'),
 ('종사자', 'NNG'),
 ('전세자금', 'NNG'),
 ('재산처분', 'NNG'),
 ('중증질환', 'NNG'),
 ('결식어르신', 'NNG'),
 ('창업보육센터', 'NNG'),
 ('청년인턴', 'NNG'),
 ('교재비', 'NNG'),
 ('선취업', 'NNG'),
 ('무료셔틀버스', 'NNG'),
 ('정서적', 'NNG'),
 ('저소득국민', 'NNG'),
 ('기타농업', 'NNG'),
 ('정착지원', 'NNG'),
 ('노인세대', 'NNG'),
 ('자동계좌이체', 'NNG'),
 ('문화예술공간', 'NNG'),
 ('민관', 'NNG'),
 ('전용주차', 'NNG'),
 ('직업훈련비', 'NNG'),
 ('위안부할머니', 'NNG'),
 ('임대지원', 'NNG'),
 ('방사성페기물', 'NNG'),
 ('녹색도시', 'NNG'),
 ('수수료', 'NNG'),
 ('차량', 'NNG'),
 ('방문상담', 'NNG'),
 ('생애주기별', 'NNG'),
 ('예절교실', 'NNG'),
 ('센터운영지원', 'NNG'),
 ('일반실', 'NNG'),
 ('직장체험', 'NNG'),
 ('일손부족', 'NNG'),
 ('여성안심택배서비스', 'NNG'),
 ('요양환자', 'NNG'),
 ('문화체험비', 'NNG'),
 ('위생

In [ ]:
# 정상적으로 사용자 사전을 인식하는지 테스트
len(test), len(final_userdic), set([tag for _, tag in test])

(2507, 2507, {'NNG'})

In [ ]:
def clean_text(text_raw):
    """
    문자열 전처리 함수.
    정규표현식을 사용해 한글/숫자/영어에 해당되지 않는 문자열을 제거
    """
    text_clean = re.sub("[^가-힣A-Za-z\\d\\s]", " ", text_raw).lower()
    return text_clean

def pos(text_raw):
    """
    형태소 분석기를 이용해 텍스트에서 명사 및 고유명사만 추출
    """
    text = clean_text(text_raw)
    # 형태소 분석기를 이용해 명사만 추출
    result = []
    for token, tag in komoran.pos(text):
        if tag == 'NNG' or tag == 'NNP':  # 명사 및 고유명사만 추출
            result.append(token)

    return " ".join(result)

# def prepro_tag(tag):
#     """
#     태그 데이터 전처리 함수
#         - 주어진 태그데이터를 하나의 문자열로 치환
#     """
#     return clean_text(" ".join(tag))

### 서비스명 품사 태깅

In [ ]:
# 품사 태깅 전, 단어 간 공백 처리

pat = ['북한\s?이탈\s?주민','새터\s?민',
       '민주\s?유공자','독립\s?유공자',
       '국가\s?유공자','특수\s?임무\s?유공자',
       '참전\s?유공자','518\s?민주화?\s?유공자',
       '518\s?유공자',
       '기초\s?생활\s?수급자','기초\s?생활\s?보장\s?수급자',
       '기초\s?생활\s?수급\s?가구','국민\s?기초\s?생활\s?대상자',
       '국민\s?기초\s?생활\s?수급자','기초\s?수급자',
       '기초\s?수급\s?대상자','국민\s?기초\s?생활\s?보장\s?수급자',
       '국민\s?기초\s?생활\s?수급자','기초\s?수급\s?가구',
       '기초\s?생활\s?보장\s?수급\s?가구','기초\s?생활\s?보급\s?수급자',
       '중증\s?장애인','경증\s?장애인',
       '청각\s?장애인','시각\s?장애인',
       '신장\s?장애인','발달\s?장애인',
       '지체\s?장애인','정신\s?장애인',
       '척수\s?장애인','지적\s?장애인',
       '자폐\s?장애인','장애\s?연금',
       '장애\s?정도\s?심한\s?장애인',
       '심한\s?장애','심하지\s?않은\s?장애',
       '다문화\s?가족?정?구?','한부모\s?가족?정?구?',
       '보훈\s?가정?족?구?','다자녀\s?가구?족?정?',
       '입양\s?가정?족?구?','조손\s?가정?족?구?',
       '교통\s?약자',
       '상수도\s?요금','수도\s?요금',
       '공영\s?주차장',
       '푸드\s?뱅크','푸드\s?마켓',
       '자살\s?위험군',
       '의료\s?급여','생계\s?급여','주거\s?급여','교육\s?급여',
       '실업\s?급여','퇴직\s?급여','생활\s?급여','보험\s?급여',
       '생계\s?급여','해산\s?급여','시설\s?급여','구직\s?급여',
       '요양\s?급여','장제\s?급여','재가\s?급여',
       '도로명\s?주소',
       '코로나\s?19',
       '뇌\s?병\s?변\s?장애인',
       '사회적\s?배려자','사회적\s?약자',
       '정보\s?소외\s?계층',
       '공공\s?도서관','공공\s?기관',
       '임대\s?주택','임대\s?아파트',
       '건강\s?검진']
       
repl =['북한이탈주민','북한이탈주민',
       '민주유공자','독립유공자',
       '국가유공자','특수임무유공자',
       '참전유공자','518민주유공자',
       '518민주유공자',
       '기초생활수급자','기초생활수급자',
       '기초생활수급자','기초생활수급자',
       '기초생활수급자','기초생활수급자',
       '기초생활수급자','기초생활수급자',
       '기초생활수급자','기초생활수급자',
       '기초생활수급자','기초생활수급자',
       '중증장애인','경증장애인',
       '청각장애인','시각장애인',
       '신장장애인','발달장애인',
       '지체장애인','정신장애인',
       '척수장애인','지적장애인',
       '자폐장애인','장애연금',
       '중증장애인',
       '중증장애','경증장애',
       '다문화가족','한부모가족',
       '보훈가족','다자녀가구',
       '입양가족','조손가족',
       '교통약자',
       '상수도요금','수도요금',
       '공영주차장',
       '푸드뱅크','푸드마켓',
       '자살위험군',
       '의료급여','생계급여','주거급여','교육급여',
       '실업급여','퇴직급여','생활급여','보험급여',
       '생계급여','해산급여','시설급여','구직급여',
       '요양급여','장제급여','재가급여',
       '도로명주소',
       '코로나19',
       '뇌병변장애인',
       '사회적약자','사회적약자',
       '정보소외계층',
       '공공도서관','공공기관',
       '임대주택','임대아파트',
       '건강검진']

In [ ]:
list(zip(pat,repl))

[('북한\\s?이탈\\s?주민', '북한이탈주민'),
 ('새터\\s?민', '북한이탈주민'),
 ('민주\\s?유공자', '민주유공자'),
 ('독립\\s?유공자', '독립유공자'),
 ('국가\\s?유공자', '국가유공자'),
 ('특수\\s?임무\\s?유공자', '특수임무유공자'),
 ('참전\\s?유공자', '참전유공자'),
 ('518\\s?민주화?\\s?유공자', '518민주유공자'),
 ('518\\s?유공자', '518민주유공자'),
 ('기초\\s?생활\\s?수급자', '기초생활수급자'),
 ('기초\\s?생활\\s?보장\\s?수급자', '기초생활수급자'),
 ('기초\\s?생활\\s?수급\\s?가구', '기초생활수급자'),
 ('국민\\s?기초\\s?생활\\s?대상자', '기초생활수급자'),
 ('국민\\s?기초\\s?생활\\s?수급자', '기초생활수급자'),
 ('기초\\s?수급자', '기초생활수급자'),
 ('기초\\s?수급\\s?대상자', '기초생활수급자'),
 ('국민\\s?기초\\s?생활\\s?보장\\s?수급자', '기초생활수급자'),
 ('국민\\s?기초\\s?생활\\s?수급자', '기초생활수급자'),
 ('기초\\s?수급\\s?가구', '기초생활수급자'),
 ('기초\\s?생활\\s?보장\\s?수급\\s?가구', '기초생활수급자'),
 ('기초\\s?생활\\s?보급\\s?수급자', '기초생활수급자'),
 ('중증\\s?장애인', '중증장애인'),
 ('경증\\s?장애인', '경증장애인'),
 ('청각\\s?장애인', '청각장애인'),
 ('시각\\s?장애인', '시각장애인'),
 ('신장\\s?장애인', '신장장애인'),
 ('발달\\s?장애인', '발달장애인'),
 ('지체\\s?장애인', '지체장애인'),
 ('정신\\s?장애인', '정신장애인'),
 ('척수\\s?장애인', '척수장애인'),
 ('지적\\s?장애인', '지적장애인'),
 ('자폐\\s?장애인', '자폐장애인'),
 ('장애\\s?연금', '장애연금'),
 

In [ ]:
def preprocess_word(text):
  result = text
  for pattern, replace in zip(pat, repl):
    result = re.sub(pattern, replace, result)
  return result

In [ ]:
title_sub = title.apply(preprocess_word)

In [ ]:
# print(preprocess_word('''
# 북한 이탈 주민,북한이탈 주민,북한 이탈주민
# 새터 민
# 국가 유공자,독립 유공자
# 민주 유공자,민주화 운동
# 기초 생활 수급자,기초생활 수급자,기초 생활수급자,기초생활보장 수급자,기초 생활 보장 수급자
# 기초생활보장 수급자
# 중증 장애인
# 청각 장애인
# 시각 장애인
# 신장 장애인
# 발달 장애인
# 다문화 가족, 한부모 가족, 보훈 가족
# 다문화 가정
# 교통 약자
# 상수도 요금, 수도 요금
# 공영 주차장
# 푸드 뱅크,푸드 마켓
# 자살 위험군
# 의료 급여, 생계 급여, 주거 급여, 교육 급여,실업 급여
# 도로명 주소
# 다자녀 가구
# 코로나 19
# 뇌 병 변 장애인
# 사회적 배려자
# 정보 소외계층
# 공공 도서관,공공 기관
# 임대 주택,임대 아파트
# 건강 검진'''))

In [ ]:
title_n = title_sub.apply(pos)

### 서비스 목적 품사 태깅

In [ ]:
purpose_sub = purpose.apply(preprocess_word)

In [ ]:
purpose_sub = purpose_sub.apply(lambda x:x.replace('\n','').replace('\t','').replace('\r','').replace('  ',' '))

In [ ]:
purpose_n = purpose_sub.copy()
fail_lst = []

for i in purpose_sub.index:
  try:
    purpose_n.loc[i] = pos(purpose_sub.loc[i])
  except:
    fail_lst.append(i)
    fail_lst.append(purpose_sub.loc[i])

In [ ]:
purpose.loc[30876]

'.'

In [ ]:
fail_lst

[30876, '.']

In [ ]:
purpose_n.loc[30876]=''

In [ ]:
#title에 대해 명사 추출
title_n

정책ID
1        유치원 유아 학비 지원 누리과정 운영
2                   근로 자녀 장려금
3                    월세 자금 보증
4              대한민국 운영자금 산업은행
5            친환경 에너지 절감 장비 보급
                 ...         
88482            피해 보전 직불금 지원
88483             이혼 위기 가족 지원
88484           여성 일 센터 운영 지원
88485              취학 아동 실명예방
88486        시흥 갯골 캠핑장 이용료 감면
Name: 서비스명, Length: 88486, dtype: object

In [ ]:
#purpose에 대해 명사 추출
purpose_n

정책ID
1        누리과정 대상 어린이집 표준 보육 과정 유치원 교육 과정 통합 지원 대상 확대 부모...
2        근로자녀 장려 세제 저소득계층 빈곤 탈출 지원 사회안전망 확충 도입 근로 소득 사업...
3                                  저소득 월세자금보증 저소득 주거 안정 제고
4                                 코로나19 피해 중소 중견기업 지원 프로그램
5                   연근해 어업인 대상 노후기관 장비 설비 체 보급 안정 조업 기반 조성
                               ...                        
88482    이행 수입 급증 가격 하락 피해 품목 생산자 가격 하락 일정 부분 피해 보전 직불금...
88483                        이혼 위기 가족 대상 가족 기능 회복 아동 복리 증진
88484    육아 가사 경력 단절 여성 대상 직업 상담 구인 구직 관리 직업 교육 인턴십 취업 ...
88485    취학 아동기 시력 검진 눈 건강관리 교육 시력 안 질환 조기 발견 치료 관리 아동 ...
88486                          시흥 갯골 캠핑장 이용 요금 할인 혜택 정보 제공
Name: 서비스목적, Length: 88486, dtype: object

# 임베딩

- 태그의 속성을 구체적인 벡터값으로 임베딩
- 임베딩된 벡터는 태그(단어)의 언어적 맥락을 반영할 수 있어야 함
    - Word2Vec 이용
    - 코싸인 유사도를 바탕으로 평가

## Word2Vec 

In [ ]:
# 하이퍼파라미터 정의

num_features = 128      # 임베딩 벡터 차원 수
min_word_count = 15     # 특정 단어의 최소 빈도수 (빈도가 적은 단어는 학습하지 않습니다!)
context = 5            # 컨텍스트 윈도우 크기 (양 옆으로 몇개의 단어를 참고할지) -> 정확히 확인
downsampling = 1e-3     # 다운샘플링 비율  

In [ ]:
# 학습 과정을 표시하기 위함
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# 명사태깅 전체 concat
noun = title_n + ' ' + purpose_n

In [ ]:
# 사전컬럼에 대한 임베딩 모델 학습

# 모델 학습용 데이터 정의
texts = []
for ele in noun:
  texts.append(ele.split())

# 모델 학습
print("Training model...")
model = word2vec.Word2Vec(texts,
                          size=num_features,
                          min_count=min_word_count,
                          window=context,
                          sample=downsampling)

# 모델 저장
model.save("./data/{}features_{}minwords_{}context".format(num_features, min_word_count, context))

2021-11-23 08:58:16,782 : INFO : collecting all words and their counts
2021-11-23 08:58:16,783 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-11-23 08:58:16,830 : INFO : PROGRESS: at sentence #10000, processed 198650 words, keeping 5929 word types
2021-11-23 08:58:16,881 : INFO : PROGRESS: at sentence #20000, processed 399700 words, keeping 7076 word types
2021-11-23 08:58:16,933 : INFO : PROGRESS: at sentence #30000, processed 603023 words, keeping 7794 word types


Training model...


2021-11-23 08:58:16,983 : INFO : PROGRESS: at sentence #40000, processed 801154 words, keeping 8609 word types
2021-11-23 08:58:17,032 : INFO : PROGRESS: at sentence #50000, processed 994799 words, keeping 9383 word types
2021-11-23 08:58:17,079 : INFO : PROGRESS: at sentence #60000, processed 1190959 words, keeping 10074 word types
2021-11-23 08:58:17,129 : INFO : PROGRESS: at sentence #70000, processed 1394240 words, keeping 10557 word types
2021-11-23 08:58:17,181 : INFO : PROGRESS: at sentence #80000, processed 1598987 words, keeping 11042 word types
2021-11-23 08:58:17,228 : INFO : collected 13470 word types from a corpus of 1765827 raw words and 88486 sentences
2021-11-23 08:58:17,229 : INFO : Loading a fresh vocabulary
2021-11-23 08:58:17,253 : INFO : effective_min_count=15 retains 4684 unique words (34% of original 13470, drops 8786)
2021-11-23 08:58:17,254 : INFO : effective_min_count=15 leaves 1733406 word corpus (98% of original 1765827, drops 32421)
2021-11-23 08:58:17,274 

## 임베딩 벡터 생성

In [ ]:
def get_embedding_vector(words, model=model, num_features=num_features):
  # 중복 단어 삭제
  words = list(set(words))

  # 출력 벡터 초기화
  feature_vector = np.zeros(num_features, dtype=np.float32)

  num_words = 0
  index2word_set = set(model.wv.index2word)    # 어휘사전

  # 어휘사전에 포함된 단어 벡터들의 평균값을 해당 텍스트의 임베딩 벡터로 사용
  for w in words:
    if w in index2word_set:
      num_words += 1
      feature_vector = np.add(feature_vector, model[w])

  feature_vector = np.divide(feature_vector, num_words)
  return feature_vector.reshape(1, -1)

In [ ]:
len(model.wv.index2word)

4684

In [ ]:
# model.wv.index2word

In [ ]:
word1 = ["고엽제"]
word2 = ["유공자"]

example1_vec = get_embedding_vector(word1)
example2_vec = get_embedding_vector(word2)

print(np.isnan(example1_vec).sum())
print(np.isnan(example1_vec).sum())

cosine_similarity(example1_vec, example2_vec)

0
0


array([[0.5024376]], dtype=float32)

In [ ]:
# cosine_similarity( get_embedding_vector(['노인', '독거']), get_embedding_vector(["유치원", "청소년"]) )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


array([[0.20358789]], dtype=float32)

# 적용

In [ ]:
concat = (content + ' ' + subject).apply(lambda x:x.replace('\n',' ').replace('\t',' ').replace('\r',' '))

In [ ]:
# 특수 단어(예:민주 유공자->민주 유공자) 등 처리
concat = concat.apply(preprocess_word)

In [ ]:
concat_n = concat.apply(pos)

In [ ]:
concat_n.to_csv("./data/지원대상_내용_noun_with_userdic.csv",encoding='CP949',index=False)

In [ ]:
title_n.to_csv("./data/서비스명_noun_with_userdic.csv",encoding='CP949',index=False)
purpose_n.to_csv("./data/서비스목적_noun_with_userdic.csv",encoding='CP949',index=False)

# 환경설정 후 이것만 돌려주세요!

In [ ]:
# 미리 학습한 모델 로드
word2vec_model = Word2Vec.load("./data/128features_15minwords_5context")

# 하이퍼파라미터 정의

num_features = 128      # 임베딩 벡터 차원 수
min_word_count = 15     # 특정 단어의 최소 빈도수 (빈도가 적은 단어는 학습하지 않습니다!)
context = 5            # 컨텍스트 윈도우 크기 (양 옆으로 몇개의 단어를 참고할지) -> 정확히 확인
downsampling = 1e-3     # 다운샘플링 비율  

def get_embedding_vector(words, model=word2vec_model, num_features=num_features):
  # 중복 단어 삭제
  words = list(set(words))

  # 출력 벡터 초기화
  feature_vector = np.zeros(num_features, dtype=np.float32)

  num_words = 0
  index2word_set = set(model.wv.index2word)    # 어휘사전

  # 어휘사전에 포함된 단어 벡터들의 평균값을 해당 텍스트의 임베딩 벡터로 사용
  for w in words:
    if w in index2word_set:
      num_words += 1
      feature_vector = np.add(feature_vector, model[w])

  feature_vector = np.divide(feature_vector, num_words)
  return feature_vector.reshape(1, -1)

In [ ]:
# 지원대상+지원내용 명사
concat_n = pd.read_csv("./data/지원대상_내용_noun_with_userdic.csv",encoding='CP949')
concat_n.index = range(1,88487)

# 서비스명 명사
title_n = pd.read_csv("./data/서비스명_noun_with_userdic.csv",encoding='CP949')
title_n.index = range(1,88487)

# 서비스목적 명사
purpose_n = pd.read_csv("./data/서비스목적_noun_with_userdic.csv",encoding='CP949')
purpose_n.index = range(1,88487)

In [ ]:
# 결측이 있는 행은 제외
title_n = title_n[title_n.서비스명.isna()==False]
purpose_n = purpose_n[purpose_n.서비스목적.isna()==False]

In [ ]:
concat_n = concat_n['0']
title_n = title_n['서비스명']
purpose_n = purpose_n['서비스목적']

In [ ]:
# 불용어
worksheet = gc.open('vector_dic_final').sheet1
rows = worksheet.get_all_values()

stop_df = pd.DataFrame.from_records(rows[1:], columns = rows[0])
stopwords = stop_df[stop_df['삭제할 단어']=='0'].단어.unique().tolist()

In [ ]:
start = 1 # 정책ID 시작
end = 10000 # 정책ID 끝

In [ ]:
# 참조컬럼이 서비스명이면 title_n을, 서비스목적이면 purpose_n으로 바꿔주세요.
IDs = sorted(list(set(range(start, end+1)) & set(title_n.index)))

title_result = pd.DataFrame(columns=['id','word','score'])

for id in tqdm(IDs):
  #참조컬럼: title_n(서비스명), purpose_n(서비스목적)에 따라 변경
  label = [x for x in title_n.loc[id].split(' ') if len(x)>1] 

  for word in set(concat_n.loc[id].split(' ')):
    # 한 글자 명사 및 불용어 제외
    if len(word)>1 and word not in stopwords:
      try:
        score= cosine_similarity(get_embedding_vector(word),
                                get_embedding_vector(label)) 
        title_result = title_result.append({'id':id,'word':word,'score':score}, ignore_index=True)                   
      except: 
        continue
        # print(id, word, np.isnan(get_embedding_vector(word)).sum())

  # 정책 id가 2000의 배수일때 마다 백업이 됨
  # 백업 주기를 조정하고 싶을 시 아래 2000을 다른 숫자로 바꾸기
  # if id % 2000 == 0: 
    
    # 경로 이름을 본인 이름으로 변경해주세요.
    # result.to_csv('./신형/result_{}_{}.csv'.format(start,id), 
    #               index=False, encoding='CP949')

title_result.score = title_result.score.map(lambda x:x[0][0])
title_result.sort_values(by=['id','score'], ascending=False, inplace=True)

title_result.to_csv('./result/result_{}_{}_title.csv'.format(start,end))

100%|██████████| 10/10 [00:00<00:00, 17.35it/s]


In [ ]:
# 참조컬럼이 서비스명이면 title_n을, 서비스목적이면 purpose_n으로 바꿔주세요.
IDs = sorted(list(set(range(start, end+1)) & set(purpose_n.index)))

purpose_result = pd.DataFrame(columns=['id','word','score'])

for id in tqdm(IDs):
  #참조컬럼: title_n(서비스명), purpose_n(서비스목적)에 따라 변경
  label = [x for x in purpose_n.loc[id].split(' ') if len(x)>1] 

  for word in set(concat_n.loc[id].split(' ')):
    # 한 글자 명사 및 불용어 제외
    if len(word)>1 and word not in stopwords:
      try:
        score= cosine_similarity(get_embedding_vector(word),
                                get_embedding_vector(label)) 
        purpose_result = purpose_result.append({'id':id,'word':word,'score':score}, ignore_index=True)                   
      except: 
        continue
        # print(id, word, np.isnan(get_embedding_vector(word)).sum())

  # 정책 id가 2000의 배수일때 마다 백업이 됨
  # 백업 주기를 조정하고 싶을 시 아래 2000을 다른 숫자로 바꾸기
  # if id % 2000 == 0: 
    
    # 경로 이름을 본인 이름으로 변경해주세요.
    # result.to_csv('./신형/result_{}_{}_purpose.csv'.format(start,id), 
    #               index=False, encoding='CP949')

purpose_result.score = purpose_result.score.map(lambda x:x[0][0])
purpose_result.sort_values(by=['id','score'], ascending=False, inplace=True)

purpose_result.to_csv('./result/result_{}_{}_purpose.csv'.format(start,end))

100%|██████████| 10/10 [00:00<00:00, 16.98it/s]


In [ ]:
total = pd.DataFrame(columns=['id','title','purpose'])

for id in tqdm(IDs):
  total = total.append({'id':id,
                        'title':title_result[title_result.id==id].head(5).word.tolist(),
                        'purpose':purpose_result[purpose_result.id==id].head(5).word.tolist(),},
                       ignore_index=True)

100%|██████████| 10/10 [00:00<00:00, 237.28it/s]


In [ ]:
total['title'] = total['title'].apply(lambda x: ','.join(x))
total['purpose'] = total['purpose'].apply(lambda x: ','.join(x))

In [ ]:
total.to_csv('./result/추출키워드_test.csv', encoding='CP949',index=False)